When you define a new container, all properties of the data model would be indexed by default.
It is your responsibility to exclude the properties that you don't need an index.
You can disable indexing on a container by passing None to the Indexing Mode.
If you keep all properties indexed, your index file might got larger than your data file.

In [1]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;
using System.Collections.ObjectModel;

Installed Packages Microsoft.Azure.Cosmos, 3.50.0 Newtonsoft.Json, 13.0.3

In [2]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring);
var db = client.GetDatabase("StackOverflow");
var postContainer = await db.GetContainer("Posts2").ReadContainerAsync();
//await db.CreateContainerIfNotExistsAsync("Posts2","/OwnerUserId", 4000);

Composite Indexes:
ORDER BY clause with two or more properties require a composite index. If you WHERE clauses have many filters, Composite Indexes can help with the performance and efficiency.

Spatial Indexes:
They increase the efficiency if you use any GeoSpatial system functions.

Vector Indexes:
They increase the efficiency if you have vectors and using VectorDistance system function.

Full Text Indexes:
They enable full text search and scoring efficiently. 

In [ ]:
postContainer.Resource.IndexingPolicy.IndexingMode = IndexingMode.None;

postContainer.Resource.IndexingPolicy.IncludedPaths.Add(new IncludedPath { Path = "/*" });

postContainer.Resource.IndexingPolicy.ExcludedPaths.Add(new ExcludedPath { Path = "/name/*" });

postContainer.Resource.IndexingPolicy.CompositeIndexes.Add(new Collection<CompositePath> { 
    new CompositePath() { Path = "/name", Order = CompositePathSortOrder.Ascending }, 
    new CompositePath() { Path = "/age", Order = CompositePathSortOrder.Descending }
    });

SpatialPath spatialPath = new SpatialPath
{
    Path = "/locations/*"
};
spatialPath.SpatialTypes.Add(SpatialType.Point);
postContainer.Resource.IndexingPolicy.SpatialIndexes.Add(spatialPath);

await db.GetContainer("Posts").ReplaceContainerAsync(postContainer.Resource);

// retrieve the container's details
ContainerResponse containerResponse = await db.GetContainer("Posts").ReadContainerAsync(new ContainerRequestOptions { PopulateQuotaInfo = true });
// retrieve the index transformation progress from the result
long indexTransformationProgress = long.Parse(containerResponse.Headers["x-ms-documentdb-collection-index-transformation-progress"]);

V3 Fluent API

In [ ]:
await client.GetDatabase("database").DefineContainer(name: "container", partitionKeyPath: "/myPartitionKey")
    .WithIndexingPolicy()
        .WithIncludedPaths()
            .Path("/*")
        .Attach()
        .WithExcludedPaths()
            .Path("/name/*")
        .Attach()
        .WithSpatialIndex()
            .Path("/locations/*", SpatialType.Point)
        .Attach()
        .WithCompositeIndex()
            .Path("/name", CompositePathSortOrder.Ascending)
            .Path("/age", CompositePathSortOrder.Descending)
        .Attach()
    .Attach()
    .CreateIfNotExistsAsync();